In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import load_dataset, Prototypes, GA_counterfactuals


In [2]:
X, X_proc, y = load_dataset("adult", preprocess=True, include_description=True)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')
Stored categorical mappings in: artifacts/categorical_encodings.json
{'age': 'Age of the individual',
 'capital-gain': 'Capital gain in USD',
 'capital-loss': 'Capital loss in USD',
 'education': 'Highest level of education achieved',
 'education-num': 'Education level as an integer',
 'fnlwgt': 'Final weight — estimation of census population',
 'hours-per-week': 'Average working hours per week',
 'marital-status': 'Marital status',
 'native-country': 'Country of origin',
 'occupation': 'Occupation',
 'race': 'Race',
 'relationship': 'Relationship status',
 'sex': 'Gender',
 'target': "Income level, '>50K' or '<=50K'",
 'workclass': 'Represents the employment status'}


/home/useradd/counterfactuals/src/data_loader/data.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds.data.rename(


In [3]:
X

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [4]:
X_proc

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country
0,25,7,0.0,0.0,40.0,3,4,3,2,1,38
1,38,9,0.0,0.0,50.0,3,2,0,4,1,38
2,28,12,0.0,0.0,40.0,1,2,0,4,1,38
3,44,10,7688.0,0.0,40.0,3,2,0,2,1,38
4,18,10,0.0,0.0,30.0,3,4,3,4,0,38
...,...,...,...,...,...,...,...,...,...,...,...
48837,27,12,0.0,0.0,38.0,3,2,5,4,0,38
48838,40,9,0.0,0.0,40.0,3,2,0,4,1,38
48839,58,9,0.0,0.0,40.0,3,6,4,4,0,38
48840,22,9,0.0,0.0,20.0,3,4,3,4,1,38


In [5]:
ARTIFACT_DIR = Path("../artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)

In [6]:
import joblib

model = joblib.load(ARTIFACT_DIR / "classification_model.pkl")

In [9]:
test = X_proc.copy()
test['outcome'] =model.predict(X_proc)

In [10]:
test

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country,outcome
0,25,7,0.0,0.0,40.0,3,4,3,2,1,38,0
1,38,9,0.0,0.0,50.0,3,2,0,4,1,38,0
2,28,12,0.0,0.0,40.0,1,2,0,4,1,38,0
3,44,10,7688.0,0.0,40.0,3,2,0,2,1,38,1
4,18,10,0.0,0.0,30.0,3,4,3,4,0,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,12,0.0,0.0,38.0,3,2,5,4,0,38,0
48838,40,9,0.0,0.0,40.0,3,2,0,4,1,38,0
48839,58,9,0.0,0.0,40.0,3,6,4,4,0,38,0
48840,22,9,0.0,0.0,20.0,3,4,3,4,1,38,0


In [8]:
instance = X_proc.iloc[0:1]
instance_outcome = y.iloc[0:1]
instance['outcome'] = instance_outcome.values

/tmp/ipykernel_4548/47571830.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instance['outcome'] = instance_outcome.values


In [9]:
instance

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country,outcome
0,25.0,7.0,0.0,0.0,40.0,3,4,3,2,1,38,0


In [10]:
pr = Prototypes(model=model, X=X_proc, y=y)

In [14]:
ga = GA_counterfactuals(X=X_proc, y=y, model=model)

In [15]:
instance

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country,outcome
0,25.0,7.0,0.0,0.0,40.0,3,4,3,2,1,38,0


In [17]:
ga._data_mutation(instance, X_proc.columns)

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country,outcome
0,233.746599,3.388753,10639.167943,862.506953,14.507872,3,4,0,4,1,38,0


In [12]:
instance

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country,outcome
0,25.0,7.0,0.0,0.0,40.0,3,4,3,2,1,38,0


In [18]:
pr.get_prototypes(
    instance,
    desired_class=1,
    k=3,
    fix_vars=['race'])


,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,sex,native_country,outcome
5953,20.0,8.0,0.0,0.0,35.0,3,4,3,2,1,38,1
31169,36.0,7.0,13550.0,0.0,40.0,3,4,1,2,1,38,1
2914,40.0,9.0,14344.0,0.0,40.0,3,4,1,2,1,38,1


In [16]:
y.name

'class'